In [24]:
import pandas as pd
import pyarrow
import os
from pyarrow import dataset as ds
def carregar_dados():
    path = "/home/leo/airflow/Projeto/data/Gold"

    if not os.path.exists(path):
        # st.warning(f"⚠️ Caminho não encontrado: {path}")
        return pd.DataFrame()
    try:
        # Usa pyarrow.dataset para ler todas as partições
        dataset = ds.dataset(path, format="parquet", partitioning="hive")
        df = dataset.to_table().to_pandas()
        # st.success("✅ Dados carregados com sucesso!")
        return df
    except Exception as e:
        # st.error(f"⚠️ Erro ao carregar dados particionados: {e}")
        return pd.DataFrame()

# 🟡 Aqui você chama a função e armazena o resultado
df = carregar_dados()


In [5]:
import os
import zipfile
import pandas as pd
from loguru import logger

KAGGLE_DATASET = "janiobachmann/bank-marketing-dataset"
OUTPUT_DIR = "/home/leo/airflow/data/kaggle_data/bank_marketing"

def baixar_dados_bank_marketing():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    logger.info("Baixando Bank Marketing dataset...")
    os.system(f"kaggle datasets download -d {KAGGLE_DATASET} -p {OUTPUT_DIR}")
    for f in os.listdir(OUTPUT_DIR):
        if f.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(OUTPUT_DIR, f), 'r') as z:
                z.extractall(OUTPUT_DIR)
            os.remove(os.path.join(OUTPUT_DIR, f))
    logger.success("Download e extração concluídos.")

def converter_parquet():
    df = pd.read_csv(os.path.join(OUTPUT_DIR, "bank.csv"))
    df['data_part'] = pd.to_datetime('now').strftime('%Y-%m-%d')
    df.to_parquet(f"{OUTPUT_DIR}/bank_marketing.parquet",index=False)
    logger.success("Bank Marketing salvo como Parquet.")

if __name__ == "__main__":
    baixar_dados_bank_marketing()
    converter_parquet()

2025-07-29 21:13:22.933 | INFO     | __main__:baixar_dados_bank_marketing:11 - Baixando Bank Marketing dataset...


... resuming from 0 bytes (145725 bytes left) ...


100%|██████████| 142k/142k [00:00<00:00, 392kB/s]
2025-07-29 21:13:26.194 | SUCCESS  | __main__:baixar_dados_bank_marketing:18 - Download e extração concluídos.


2025-07-29 21:13:26.408 | SUCCESS  | __main__:converter_parquet:24 - Bank Marketing salvo como Parquet.


ModuleNotFoundError: No module named 'pyspark'